# This notebook contains functions to plot figures involving multiple station pairs.

In [1]:
import os,random,time,pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from obspy.signal.filter import bandpass
from seisgo import noise,utils,dispersion
import pygmt as gmt

## Global parameters

In [2]:
dataroot='data_stacking'
raw=os.path.join(dataroot,"MERGED_cascadia_raw")
tnorm=os.path.join(dataroot,"MERGED_cascadia_tnorm")
raw_xz=os.path.join(dataroot,"MERGED_XZ_raw")
tnorm_xz=os.path.join(dataroot,"MERGED_XZ_tnorm")
comp="ZZ"
## Cascadia amphibious array and XZ moveout plots with all stacking methods
figformat="tiff"
figdpi=300

dlabels=["Raw","One-bit"]
cmap="seismic"
methods=["linear","robust","selective","cluster","pws","tfpws","nroot","acf"]
method_labels=["Linear","Robust","Selective","Cluster","PWS","tf-PWS","$N^{th}$-root","ACF"]
colors=["k","b","c","y","g","orange","r","m"]
figlabels=["(a)","(b)","(c)","(d)","(e)","(f)","(g)","(h)"]

## Cascadia moveout plot
### Land receivers.

In [3]:
with open('xcorr_stacks_allpair_receiverinfo_cascadia.pk','rb') as xf:
    receiver_ele_all=pickle.load(xf)
    receiver_name_all=pickle.load(xf)
with open('xcorr_stacks_allpair_cascadia.pk','rb') as xf:
    stack_all=pickle.load(xf)
    dist_all=pickle.load(xf)
    
source="7D.J33A"
receiver="TA.G03D"
network="cascadia"
pair=source+"_"+receiver
datadir_raw=os.path.join(raw,source)
dfile_raw=datadir_raw+"/"+pair+".h5"
cdata_raw=noise.extract_corrdata(dfile_raw,pair=pair,comp=comp)[pair][comp]

#manually created file list including all "source" station (either as a source or as a receiver)
dfilelist_raw=raw+"/all_"+source+".txt"
dfilelist_tnorm=tnorm+"/all_"+source+".txt"
figsize=(12,5.5)
lag=250
scale=5
savefig=True
freqmin,freqmax=[0.1,0.4]#[0.04,0.4]
vmin=2 
vmax=4.5
snr_window_extend=100
snr_min=1.0
maxlag=cdata_raw.lag
dt=cdata_raw.dt
tvec=np.arange(-maxlag,maxlag+0.5*dt,dt)
dist=[50,300]
for i in range(len(dlabels)):
    fig=plt.figure(figsize=figsize,facecolor='w')
    fig.suptitle("Land receivers from virtual source: "+source+": "+dlabels[i]\
                 +": "+str(freqmin)+"-"+str(freqmax)+" Hz",fontsize=14)
    for j,m in enumerate(method_labels):
        plt.subplot(2,4,j+1)
        for k in range(stack_all[dlabels[i]].shape[1]):
            if dist_all[dlabels[i]][k] >= dist[0] and dist_all[dlabels[i]][k] <= dist[1]:
                dp = bandpass(stack_all[dlabels[i]][j,k,:],freqmin,freqmax,1/dt,corners=4, zerophase=True)
                plt.plot(tvec,dist_all[dlabels[i]][k]+scale*dp/np.max(np.abs(dp)),
                                 color=[.2,.2,.2],linewidth=0.75,alpha=1)
        # plot signal window
        plt.plot([snr_window_extend + dist[0]/vmin,snr_window_extend+ dist[1]/vmin],dist,'r',lw=1)
        plt.plot([dist[0]/vmax,dist[1]/vmax],dist,'r',lw=1)
        
        plt.plot([-snr_window_extend - dist[0]/vmin,-snr_window_extend- dist[1]/vmin],dist,'b--',lw=1)
        plt.plot([-dist[0]/vmax,-dist[1]/vmax],dist,'b--',lw=1)
        
        plt.xlim([-lag,lag])
        plt.ylim([dist[0]-0.5*scale,dist[1]+0.5*scale])
        if j>=4:
            plt.xlabel('lag time (s)',fontsize=12)
        if j==0 or j==4:
            plt.ylabel('distance (km)',fontsize=12)
#         plt.text(-0.9*lag,dist[0]+20,m,backgroundcolor='w',fontsize=13)
        plt.title('%s %s' % (figlabels[j],m),
                        fontsize=14,loc="left",pad=9)
        plt.plot((0,0),dist,'k-')
    plt.tight_layout()
    if savefig:
        plt.savefig("xcorr_stacks_allpair_moveout_cascadia_%s.%s"%(dlabels[i],figformat),dpi=figdpi)
        plt.close()
    else:
        plt.show()

### OBS receivers

In [4]:
with open('xcorr_stacks_allpair_receiverinfo_cascadia.pk','rb') as xf:
    receiver_ele_all=pickle.load(xf)
    receiver_name_all=pickle.load(xf)
with open('xcorr_stacks_allpair_cascadia.pk','rb') as xf:
    stack_all=pickle.load(xf)
    dist_all=pickle.load(xf)
    
source="7D.J33A"
receiver="TA.G03D"
network="cascadia"
pair=source+"_"+receiver
datadir_raw=os.path.join(raw,source)
dfile_raw=datadir_raw+"/"+pair+".h5"
cdata_raw=noise.extract_corrdata(dfile_raw,pair=pair,comp=comp)[pair][comp]

#manually created file list including all "source" station (either as a source or as a receiver)
dfilelist_raw=raw+"/all_"+source+".txt"
dfilelist_tnorm=tnorm+"/all_"+source+".txt"
figsize=(12,5.5)
lag=550
scale=6
savefig=True
freqmin,freqmax=[0.1,0.4]#[0.04,0.4]
vmin_obs=0.5 
vmax_obs=1.0
snr_window_extend_obs=60
maxlag=cdata_raw.lag
dt=cdata_raw.dt
tvec=np.arange(-maxlag,maxlag+0.5*dt,dt)
dist_obs=[50,240]
for i in range(len(dlabels)):
    fig=plt.figure(figsize=figsize,facecolor='w')
    fig.suptitle("OBS receivers from virtual source: "+source+": "+dlabels[i]\
                 +": "+str(freqmin)+"-"+str(freqmax)+" Hz",fontsize=14)
    for j,m in enumerate(method_labels):
        plt.subplot(2,4,j+1)
        for k in range(stack_all[dlabels[i]].shape[1]):
            if dist_all[dlabels[i]][k] >= dist_obs[0] and dist_all[dlabels[i]][k] <= dist_obs[1]:
                dp = bandpass(stack_all[dlabels[i]][j,k,:],freqmin,freqmax,1/dt,corners=4, zerophase=True)
                if receiver_ele_all[dlabels[i]][k] < 0:
                    plt.plot(tvec,dist_all[dlabels[i]][k]+scale*dp/np.max(np.abs(dp)),
                                 color=[0.2,0.2,0.2],linewidth=0.75,alpha=1)
        # plot signal window
        plt.plot([snr_window_extend_obs + dist_obs[0]/vmin_obs,snr_window_extend_obs+ dist_obs[1]/vmin_obs],dist_obs,'r',lw=1)
        plt.plot([dist_obs[0]/vmax_obs,dist_obs[1]/vmax_obs],dist_obs,'r',lw=1)
        
        plt.plot([-snr_window_extend_obs - dist_obs[0]/vmin_obs,-snr_window_extend_obs- dist_obs[1]/vmin_obs],dist_obs,'b--',lw=1)
        plt.plot([-dist_obs[0]/vmax_obs,-dist_obs[1]/vmax_obs],dist_obs,'b--',lw=1)
        
        plt.xlim([-lag,lag])
        plt.ylim([dist_obs[0]-0.5*scale,dist_obs[1]+0.5*scale])
        if j>=4:
            plt.xlabel('lag time (s)',fontsize=12)
        if j==0 or j==4:
            plt.ylabel('distance (km)',fontsize=12)
        plt.title('%s %s' % (figlabels[j],m),
                        fontsize=14,loc="left",pad=9)
        plt.plot((0,0),dist_obs,'k-')
    plt.tight_layout()
    if savefig:
        plt.savefig("xcorr_stacks_allpair_moveout_cascadia_%s_OBS.%s"%(dlabels[i],figformat),dpi=figdpi)
        plt.close()
    else:
        plt.show()

### XZ moveout plot

In [5]:
XZ_source="XZ.A02"
network_xz="XZ"
with open('xcorr_stacks_allpair_XZ.pk','rb') as xf:
    stack_all_xz=pickle.load(xf)
    dist_all_xz=pickle.load(xf)
flist_xz=[utils.get_filelist(raw_xz+"/"+XZ_source,"h5"),
          utils.get_filelist(tnorm_xz+"/"+XZ_source,"h5")]
i=0
cdict_temp=noise.extract_corrdata(flist_xz[0][0].replace("\n",""),comp=comp)
cdata0=cdict_temp[list(cdict_temp.keys())[0]][comp]
npts_xz=cdata0.data.shape[1] #get number of points.
maxlag_xz=cdata0.lag
dt_xz=cdata0.dt

In [6]:
figsize=(12,5.5)
lag=160
scale=8
savefig=True
freqmin_xz,freqmax_xz=[0.1,.4]#[0.04,0.4]
snr_window_extend_xz=20
if lag>maxlag_xz:raise ValueError('lag excceds maxlag!')
tvec_xz=np.arange(-maxlag_xz,maxlag_xz+0.5*dt_xz,dt_xz)
dist_xz=[10,240]
vmin_xz=2
vmax_xz=3.7
for i in range(len(dlabels)):
    fig=plt.figure(figsize=figsize,facecolor='w')
    fig.suptitle("Virtual source: "+XZ_source+": "+dlabels[i]\
                 +": "+str(freqmin_xz)+"-"+str(freqmax_xz)+" Hz",fontsize=14)
    for j,m in enumerate(method_labels):
        plt.subplot(2,4,j+1)
        for k in range(stack_all_xz[dlabels[i]].shape[1]):
            if dist_all_xz[dlabels[i]][k] >= dist_xz[0] and dist_all_xz[dlabels[i]][k] <= dist_xz[1]:
                dp = bandpass(stack_all_xz[dlabels[i]][j,k,:],freqmin_xz,freqmax_xz,1/dt_xz,corners=4, zerophase=True)
                plt.plot(tvec_xz,dist_all_xz[dlabels[i]][k]+scale*dp/np.max(np.abs(dp)),
                             color=[.2,.2,.2],linewidth=0.75,alpha=1)
        # plot signal window
        plt.plot([snr_window_extend_xz + dist_xz[0]/vmin_xz,snr_window_extend_xz+ dist_xz[1]/vmin_xz],
                 dist_xz,'r',lw=1)
        plt.plot([dist_xz[0]/vmax_xz,dist_xz[1]/vmax_xz],dist_xz,'r',lw=1)
        
        plt.plot([-snr_window_extend_xz - dist_xz[0]/vmin_xz,-snr_window_extend_xz- dist_xz[1]/vmin_xz],
                 dist_xz,'b--',lw=1)
        plt.plot([-dist_xz[0]/vmax_xz,-dist_xz[1]/vmax_xz],dist_xz,'b--',lw=1)
        
        plt.xlim([-lag,lag])
        plt.ylim([dist_xz[0]-0.5*scale,dist_xz[1]+0.5*scale])
        if j>=4:
            plt.xlabel('lag time (s)',fontsize=12)
        if j==0 or j==4:
            plt.ylabel('distance (km)',fontsize=12)
#         plt.text(-0.9*lag,dist_xz[0]+20,m,backgroundcolor='w',fontsize=13)
        plt.title('%s %s' % (figlabels[j],m),
                        fontsize=14,loc="left",pad=9)
        plt.plot((0,0),dist_xz,'k-')
    plt.tight_layout()
    if savefig:
        plt.savefig("xcorr_stacks_allpair_moveout_XZ_%s.%s"%(dlabels[i],figformat),dpi=figdpi)
        plt.close()
    else:
        plt.show()

### Mean SNR for both datasets, with error bars
Shorter noise windows will be used when the window exceeds the length of the time-series. This is because some OBS pairs have late arrivals but the total lenght is not enough to have an equal-sized noise window.

In [7]:
figsize=(12,7)
savefig=True
barx=np.arange(0,len(method_labels))+1
plt.figure(figsize=figsize,facecolor='w')
markersize=8
db=True
for i in range(len(dlabels)):
    #
    snr_all_land=[]
    snr_all_obs=[]
    for j,m in enumerate(method_labels): 
        snr_temp_land=[]
        snr_temp_obs=[]
        for k in range(stack_all[dlabels[i]].shape[1]):
            #obs receivers.
            if receiver_ele_all[dlabels[i]][k]<0:
                if dist_all[dlabels[i]][k] >= dist_obs[0] and dist_all[dlabels[i]][k] <= dist_obs[1]:
                    dp = bandpass(stack_all[dlabels[i]][j,k,:],freqmin,freqmax,1/dt,corners=4, zerophase=True)
                    snr=utils.get_snr(dp,tvec,dist_all[dlabels[i]][k],vmin_obs,vmax_obs,offset=30,
                                     extend=snr_window_extend_obs,db=db,shorten_noise=True)
                    snr_temp_obs.append(snr)
            else: #land receivers
                if dist_all[dlabels[i]][k] >= dist[0] and dist_all[dlabels[i]][k] <= dist[1]:
                    dp = bandpass(stack_all[dlabels[i]][j,k,:],freqmin,freqmax,1/dt,corners=4, zerophase=True)
                    snr=utils.get_snr(dp,tvec,dist_all[dlabels[i]][k],vmin,vmax,offset=30,
                                     extend=snr_window_extend,db=db)
                    snr_temp_land.append(snr)
            
        snr_all_land.append(snr_temp_land)
        snr_all_obs.append(snr_temp_obs)
        
    snr_all_land=np.array(snr_all_land)
    snr_all_obs=np.array(snr_all_obs)
    
    ####
    snr_all_xz=[]
    for j,m in enumerate(method_labels): 
        snr_temp=[]
#         dist_plot=[]
        for k in range(stack_all_xz[dlabels[i]].shape[1]):
            if dist_all_xz[dlabels[i]][k] >= dist_xz[0] and dist_all_xz[dlabels[i]][k] <= dist_xz[1]:
                dp = bandpass(stack_all_xz[dlabels[i]][j,k,:],freqmin_xz,freqmax_xz,1/dt,corners=4, zerophase=True)
                snr=utils.get_snr(dp,tvec_xz,dist_all_xz[dlabels[i]][k],vmin_xz,vmax_xz,offset=30,
                                 extend=snr_window_extend_xz,db=db)
                snr_temp.append(snr)
#                 dist_plot.append(dist_all[dlabels[i]][k])
        snr_all_xz.append(snr_temp)
    snr_all_xz=np.array(snr_all_xz)

    ####plot
    plt.subplot(2,2,i+1)
    plt.errorbar(barx-0.17,np.nanmean(snr_all_land[:,:,0],axis=1),yerr=np.nanstd(snr_all_land[:,:,0],axis=1),
                 fmt="o",markersize=6,markeredgecolor="tab:blue",markerfacecolor="None",markeredgewidth=1,
                 label="neg.-land",ls='none',capsize=3,color="tab:blue",linewidth=1)
    plt.errorbar(barx-0.07,np.nanmean(snr_all_land[:,:,1],axis=1),yerr=np.nanstd(snr_all_land[:,:,1],axis=1),
                 fmt="^",markersize=6,markeredgecolor="tab:orange",markerfacecolor="None",markeredgewidth=1,
                 label="pos.-land",ls='none',capsize=3,color="tab:orange",linewidth=1)
    plt.errorbar(barx+0.07,np.nanmean(snr_all_obs[:,:,0],axis=1),yerr=np.nanstd(snr_all_obs[:,:,0],axis=1),
                 fmt="o",markersize=5,markeredgecolor="tab:blue",markerfacecolor="tab:blue",markeredgewidth=1,
                 label="neg.-OBS",ls='none',capsize=3,color="tab:blue",linewidth=1)
    plt.errorbar(barx+0.17,np.nanmean(snr_all_obs[:,:,1],axis=1),yerr=np.nanstd(snr_all_obs[:,:,1],axis=1),
                 fmt="^",markersize=5,markeredgecolor="tab:orange",markerfacecolor="tab:orange",markeredgewidth=1,
                 label="pos.-OBS",ls='none',capsize=3,color="tab:orange",linewidth=1)
    
    if i==0:plt.legend(ncol=2,fontsize=12,loc="best",columnspacing=.5,borderpad=0.3,labelspacing=0.3)
    plt.xticks(barx,labels=method_labels,fontsize=13,rotation=30)
    plt.yticks(fontsize=12)
    if i==0:plt.ylabel('SNR (dB)',fontsize=13)
    plt.ylim([-15,28])
#     plt.yscale('log')
    plt.grid(b=True,which='both')
    plt.title('%s Amphibious: %s' % (figlabels[i],dlabels[i]),
                    fontsize=14,loc="left",pad=9)
    
    plt.subplot(2,2,i+3)
    plt.errorbar(barx-0.08,np.nanmean(snr_all_xz[:,:,0],axis=1),yerr=np.nanstd(snr_all_xz[:,:,0],axis=1),
                 fmt="o",markersize=6,markerfacecolor="None",markeredgewidth=1,
                 label="negative",ls='none',capsize=3,linewidth=1)
    plt.errorbar(barx+0.08,np.nanmean(snr_all_xz[:,:,1],axis=1),yerr=np.nanstd(snr_all_xz[:,:,1],axis=1),
                 fmt="^",markersize=6,markerfacecolor="None",markeredgewidth=1,
                 label="positive",ls='none',capsize=3,linewidth=1)

    if i==0:plt.legend(ncol=2,fontsize=12,loc="upper left",columnspacing=.5)
    plt.grid("both")
    plt.xticks(barx,labels=method_labels,fontsize=13,rotation=30)
    plt.yticks(fontsize=12)
    plt.ylim([-2,40])
    if i==0:plt.ylabel('SNR (dB)',fontsize=13)
    plt.title('%s XZ: %s' % (figlabels[i+2],dlabels[i]),
                    fontsize=14,loc="left",pad=9)
plt.tight_layout()
if savefig:
    plt.savefig("xcorr_stacks_allpair_snr_meanerr."+figformat,dpi=figdpi)
    plt.close()
else:
    plt.show()

Noise window end [7223]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7178]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7244]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7289]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7481]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7280]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7305]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7398]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7775]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7623]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7544]is larger than the data length [7000]. Force it to stop at the end.

Noise window end [7223]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7178]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7244]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7289]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7481]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7280]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7305]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7398]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7775]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7623]is larger than the data length [7000]. Force it to stop at the end.
Noise window end [7544]is larger than the data length [7000]. Force it to stop at the end.

## Extract and plot XZ peak amplitudes within the signal window

In [11]:
from scipy.optimize import curve_fit
def expfunc(x, a,b):
    return  a*np.exp(-b * x)

### Extract peak absolute amplitudes.

In [12]:
###amplitudes
lag=200
if not lag:lag=maxlag_xz
if lag>maxlag_xz:raise ValueError('lag excceds maxlag!')
tvec_xz=np.arange(-maxlag_xz,maxlag_xz+0.5*dt_xz,dt_xz)
freqmin,freqmax=[0.1,0.4]
vmin_xz=2
vmax_xz=3.7
peakamp_xz=dict()
peaktt_xz=dict()
halfn_xz=int(len(tvec_xz)/2)
for i in range(len(dlabels)):
    ntrace=stack_all_xz[dlabels[i]].shape[1]
    peakamp_xz[dlabels[i]]=np.ndarray((len(methods),ntrace,2)) #negative and positive sides.
    peaktt_xz[dlabels[i]]=np.ndarray((len(methods),ntrace,2)) #negative and positive sides.
    
    for j,m in enumerate(method_labels):
        for k in range(ntrace):
            dp = bandpass(stack_all_xz[dlabels[i]][j,k,:],freqmin,freqmax,1/dt,corners=4, zerophase=True)
            d0=dist_all_xz[dlabels[i]][k]
            sig_idx_p=[int(d0/vmax_xz/dt_xz)+halfn_xz,
                       halfn_xz+int(d0/vmin_xz/dt + snr_window_extend_xz/dt)]
            sig_idx_n=[len(tvec_xz) - sig_idx_p[1],len(tvec_xz) - sig_idx_p[0]]
            dp=np.abs(dp)
            peakamp_xz[dlabels[i]][j,k,:]=[np.max(dp[sig_idx_n[0]-1:sig_idx_n[1]+1]),
                                           np.max(dp[sig_idx_p[0]-1:sig_idx_p[1]+1])]
            ttidx=[np.argmax(dp[sig_idx_n[0]-1:sig_idx_n[1]+1]),
                   np.argmax(dp[sig_idx_p[0]-1:sig_idx_p[1]+1])]
            peaktt_xz[dlabels[i]][j,k,:]=[tvec_xz[ttidx[0]],tvec_xz[ttidx[1]]]
        

### Plot peak absolute amplitudes and find the best exponential fit.

In [13]:
from scipy.stats import linregress
dist_xz=[vmax_xz/freqmin,240]
ampfit_all_xz=np.ndarray((len(dlabels),len(methods),2,2))
savefig=True
figsize=(13,6.7)
ref_exp_pow1=[0.0051,0.0077] #Pireto et al., 2009, for period at 5 s
ref_exp_pow2=[0.001,0.003] #Mitchell, 1995, at around 6-10 s, for tectonically active regions.
ref_exp_pow3=0.014 #Viens et al., 2017, at 3-10 s, no std given
dist_fit=np.arange(dist_xz[0],dist_xz[1],1)
for i in range(len(dlabels)):
    fig=plt.figure(figsize=figsize,facecolor='w')
    fig.suptitle("Peak absolute amplitudes: "+dlabels[i],fontsize=14)
    dist_idx=np.where((dist_all_xz[dlabels[i]]>= dist_xz[0]) & (dist_all_xz[dlabels[i]]<= dist_xz[1]))[0]
    for j,m in enumerate(method_labels):
        plt.subplot(2,4,j+1)
        # 
        dist_subset=dist_all_xz[dlabels[i]][dist_idx]
        
        for k in range(2):
            if k==0: continue #skip negative side
            #correct for SQRT(distance) to account for geometrical spreading.
            amp_subset=np.multiply(np.sqrt(dist_subset),peakamp_xz[dlabels[i]][j,dist_idx,k])#

            #fit with linear regression after converting to natural log scale.
            amptemp=np.log(amp_subset)
            mask = ~np.isnan(amptemp)
            res0=linregress(dist_subset[mask],amptemp[mask])
            
            #remove extreme outliers
            amptemp00 = amptemp - (res0.intercept + res0.slope*dist_subset)
            amp_lb=np.nanmean(amptemp00) - 1.*np.nanstd(amptemp00)
            amp_ub=np.nanmean(amptemp00) + 1.*np.nanstd(amptemp00)
            goodidx=np.where((amptemp00 >= amp_lb) & (amptemp00 <= amp_ub))[0]
            ampfinal=amptemp[goodidx]
            distfinal=dist_subset[goodidx]
            
            res=linregress(distfinal,ampfinal)
            yl=expfunc(dist_fit,np.exp(res.intercept),-1*(res.slope-1.*res.stderr))
            yu=expfunc(dist_fit,np.exp(res.intercept),-1*(res.slope+1.*res.stderr))
            if j==0:
                plt.fill_between(dist_fit,expfunc(dist_fit,np.exp(res.intercept),ref_exp_pow1[1]),
                                 expfunc(dist_fit,np.exp(res.intercept),ref_exp_pow1[0]),
                                 color=[0.5,0.5,0.5],label="P2009",alpha=0.7)
                plt.fill_between(dist_fit,expfunc(dist_fit,np.exp(res.intercept),ref_exp_pow2[1]),
                                 expfunc(dist_fit,np.exp(res.intercept),ref_exp_pow2[0]),
                                 color='b',label="M1995",alpha=0.4)
#                 plt.plot(dist_fit,expfunc(dist_fit,np.exp(res.intercept),ref_exp_pow3),
#                                  '-',
#                                  color='g',lw=4,label="V2017",alpha=0.6)
                
                plt.fill_between(dist_fit,yl,yu,alpha=0.5,color='r',
                                 label='This fit')
            else:
                plt.fill_between(dist_fit,expfunc(dist_fit,np.exp(res.intercept),ref_exp_pow1[1]),
                             expfunc(dist_fit,np.exp(res.intercept),ref_exp_pow1[0]),
                             color=[0.5,0.5,0.5],alpha=0.7)
                plt.fill_between(dist_fit,expfunc(dist_fit,np.exp(res.intercept),ref_exp_pow2[1]),
                                 expfunc(dist_fit,np.exp(res.intercept),ref_exp_pow2[0]),
                                 color='b',alpha=0.4)
#                 plt.plot(dist_fit,expfunc(dist_fit,np.exp(res.intercept),ref_exp_pow3),
#                                  '-',color='g',lw=4,alpha=0.6)
                plt.fill_between(dist_fit,yl,yu,alpha=0.5,color='r')
            
            plt.scatter(distfinal,amp_subset[goodidx],s=15,color='k',alpha=0.7)
            if i==0 and j==4:
                plt.text(0.95*np.max(dist_xz),1.1*np.nanmax(amp_subset[goodidx]),
                     '$\u03B1=%.4f$\n$\u03C3=%.4f$'%(-res.slope,res.stderr),color='r',alpha=1,ha="right",
                    fontsize=12)
            else:
                plt.text(0.95*np.max(dist_xz),0.85*np.nanmax(amp_subset[goodidx]),
                     '$\u03B1=%.4f$\n$\u03C3=%.4f$'%(-res.slope,res.stderr),color='r',alpha=1,ha="right",
                    fontsize=12)

        if j>=4: plt.xlabel('distance (km)',fontsize=12)
        if j==0 or j==4: plt.ylabel('peak amplitude',fontsize=12)
        plt.xlim([dist_xz[0]-2,np.max(dist_subset)])
        
#         plt.yscale('log')
        if j==0: plt.legend(fontsize=10.5,loc="lower left",edgecolor='none',
                   numpoints=5,borderpad=0.2)
        plt.title('%s %s' % (figlabels[j],m),
                            fontsize=14,loc="left",pad=15)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.ticklabel_format(axis='y',style='scientific',scilimits=(0,0))
    plt.tight_layout()
    if savefig:
        plt.savefig("xcorr_stacks_ampfit_XZ_%s_%s-%sHz.%s"%(dlabels[i],
                                                             str(freqmin).replace(".","_"),
                                                             str(freqmax).replace(".","_"),figformat),dpi=figdpi)
        plt.close()
    else:
        plt.show()

## Station map

### Assemble station information.

In [14]:
fh=open(dfilelist_raw,'r')
flist=fh.readlines()
net=[]
sta=[]
lon=[]
lat=[]
ele=[]
for i,f in enumerate(flist):
    f0=f.replace("\n","")
    cdata=noise.extract_corrdata(raw+"/"+f0,comp=comp,dataless=True)
    pall=list(cdata.keys())
    dst=cdata[pall[0]][comp].dist
    if dst>=dist[0] and dst<=dist[1]:
        net.append(cdata[pall[0]][comp].net[0])
        net.append(cdata[pall[0]][comp].net[1])

        sta.append(cdata[pall[0]][comp].sta[0])
        sta.append(cdata[pall[0]][comp].sta[1])

        lon.append(cdata[pall[0]][comp].lon[0])
        lon.append(cdata[pall[0]][comp].lon[1])

        lat.append(cdata[pall[0]][comp].lat[0])
        lat.append(cdata[pall[0]][comp].lat[1])

        ele.append(cdata[pall[0]][comp].ele[0])
        ele.append(cdata[pall[0]][comp].ele[1])
    
#
fh.close()
stainfo_cascadia=pd.DataFrame.from_dict({"network":net,"station":sta,"longitude":lon,
                                    "latitude":lat,"elevation":ele}).drop_duplicates().reset_index()

flist_xz=utils.get_filelist(raw_xz+"/"+XZ_source,"h5")
net=[]
sta=[]
lon=[]
lat=[]
ele=[]
for i,f in enumerate(flist_xz):
    f0=f.replace("\n","")
    cdata=noise.extract_corrdata(f0,comp=comp,dataless=True)
    pall=list(cdata.keys())
    dst=cdata[pall[0]][comp].dist
    if dst>=dist_xz[0] and dst<=dist_xz[1]:
        net.append(cdata[pall[0]][comp].net[0])
        net.append(cdata[pall[0]][comp].net[1])

        sta.append(cdata[pall[0]][comp].sta[0])
        sta.append(cdata[pall[0]][comp].sta[1])

        lon.append(cdata[pall[0]][comp].lon[0])
        lon.append(cdata[pall[0]][comp].lon[1])

        lat.append(cdata[pall[0]][comp].lat[0])
        lat.append(cdata[pall[0]][comp].lat[1])

        ele.append(cdata[pall[0]][comp].ele[0])
        ele.append(cdata[pall[0]][comp].ele[1])
    #
stainfo_xz=pd.DataFrame.from_dict({"network":net,"station":sta,"longitude":lon,
                                    "latitude":lat,"elevation":ele}).drop_duplicates().reset_index()

### Plot station map with pygmt

In [15]:
region=[-128.5,-120.5,42.5,47.8]
# grid_subset = gmt.datasets.load_earth_relief(resolution="01m", region=region)
# s1=["c0.2c"]*len(stainfo_cascadia)
# s2=["t0.15c"]*len(stainfo_xz)
# s_all=s1+s2
title="station map"
style="fancy"
distance=None
projection="M3i"
xshift="6i"
frame="a2f1"
save=True
XZ_receiver="XZ.A24"
fig = gmt.Figure()
gmt.config(MAP_FRAME_TYPE=style,FORMAT_GEO_MAP="ddd")
# fig.grdimage(grid=grid_subset, projection=projection, region=region, frame=frame)
# fig.colorbar(frame=["a2000f1000", "x+lElevation", "y+lm"])
fig.coast(region=region, resolution="i",projection=projection,frame=frame,
          shorelines="0.5p,100",borders=["2/1p,black","1/1.5p,black"],
         water="lightblue",land="220")
# fig.basemap(frame='+t"'+title+'"')
fig.plot(
    x=stainfo_cascadia['longitude'],
    y=stainfo_cascadia['latitude'],
    style="t0.22c",
    pen="0.75p,red",
    label="Cascadia-amphibious"
)

fig.plot(
    x=stainfo_xz['longitude'],
    y=stainfo_xz['latitude'],
    style="c0.15c",
    pen="0.25p,255",
    color='blue',
    label="XZ-linear"
)

#search for example stations
stasub1=stainfo_cascadia[(stainfo_cascadia["network"]=="7D") & (stainfo_cascadia["station"]=="J33A")]
stasub2=stainfo_cascadia[(stainfo_cascadia["network"]=="TA") & (stainfo_cascadia["station"]=="G03D")]
stasub1_XZ=stainfo_xz[(stainfo_xz["network"]=="XZ") & (stainfo_xz["station"]==XZ_source.split(".")[1])]
stasub2_XZ=stainfo_xz[(stainfo_xz["network"]=="XZ") & (stainfo_xz["station"]==XZ_receiver.split(".")[1])]

fig.plot(
    x=stasub1['longitude'],
    y=stasub1['latitude'],
    style="c0.27c",
    pen="1p,0",
)

fig.plot(
    x=stasub2['longitude'],
    y=stasub2['latitude'],
    style="c0.27c",
    pen="1p,0",
)
fig.text(text="7D.J33A", x=stasub1['longitude']-1., y=stasub1['latitude'], 
         font="11p,Helvetica,0", fill="white")
fig.text(text="TA.G03D", x=stasub2['longitude']+0., y=stasub2['latitude']+0.3, 
         font="11p,Helvetica,0", fill="white")

fig.plot(
    x=stasub1_XZ['longitude'],
    y=stasub1_XZ['latitude'],
    style="c0.18c",
    pen="1p,green",
)

fig.plot(
    x=stasub2_XZ['longitude'],
    y=stasub2_XZ['latitude'],
    style="c0.18c",
    pen="1p,green",
)
fig.plot(x=-125.,y=43.7,style="v0.35c+eA+a30",direction=([45], [1.3]), pen="1p", color="red3")
fig.text(text=XZ_source, x=-125.5,y=43.7, font="11p,Helvetica,0", fill="white")
fig.plot(x=-121.9,y=43.7,style="v0.35c+eA+a30",direction=([135], [1.3]), pen="1p", color="red3")
fig.text(text=XZ_receiver, x=-121.5,y=43.7, font="11p,Helvetica,0", fill="white")

fig.text(text="Oregon", x=-122.,y=42.8, 
         font="11p,Helvetica,0")
fig.text(text="Washington", x=-121.7,y=46.9, 
         font="11p,Helvetica,0")
fig.text(text="Pacific", x=-127.7,y=47.6, 
         font="10p,Helvetica,0")
fig.text(text="Ocean", x=-127.7,y=47.3, 
         font="10p,Helvetica,0")
fig.legend(position="JBL+jBL+o0.1c",box=True)
if save:
    figname="stationmap_cascadia_XZ."+"pdf"
    fig.savefig(figname,dpi=figdpi)
    print('plot was saved to: '+figname)
else:
    fig.show()

plot was saved to: stationmap_cascadia_XZ.pdf


### Save selected stations into CSV files

In [16]:
lon_all=np.concatenate((stainfo_cascadia['longitude'],stainfo_xz['longitude']))
lat_all=np.concatenate((stainfo_cascadia['latitude'],stainfo_xz['latitude']))

stainfo_cascadia.to_csv("cascadia_station_used.csv",index=False)


stainfo_xz.to_csv("XZ_station_used.csv",index=False)